Detect motion with PIL based on the recipe for OpenCV in the reference. This is because installing OpenCV on Raspberry Pi is a bear.

In [50]:
from picamera import PiCamera, array
import numpy as np
from io import BytesIO
from PIL import Image, ImageFilter, ImageMorph, ImageEnhance
import time
import os

Detect motion by imaging in two takes, one after the other, and comparing pixels. Let's write a function to take a snap.

In [51]:
def take_motion_snap(width, height):
    with PiCamera() as Eye:
        time.sleep(1)
        Eye.resolution = (width, height)
        Eye.rotation = 180
        with array.PiRGBArray(camera=Eye) as Stream:
            Eye.exposure_mode = 'auto'
            Eye.awb_mode = 'auto'
            Eye.capture(Stream, format='rgb')
            return Stream.array

In [52]:
test = take_motion_snap(300, 300)
print("Got an image with width, height as {}, {}.".format(test.shape[0], test.shape[1]))
snap = Image.fromarray(test)
snap.show()

Got an image with width, height as 300, 300.


Now we have an RGB image of the specified height and width as a 3-dimensional numpy array. We want to take the difference between two snaps. Write a wrapper function to use ```take_motion_snap(w, h)```, take two snaps and return the computed difference.

In [53]:
def take_two_motion(intervalsec):
    im_one = take_motion_snap(300, 300)
    tic = time.time()
    toc = tic
    while (toc - tic) < intervalsec:
        toc = time.time()
    im_two = take_motion_snap(300, 300)
    im_diff = np.subtract(im_two, im_one)
    return im_diff


In [54]:
test_diff = take_two_motion(6)
print("Got a difference of width, height as {}, {}.".format(test_diff.shape[0], test_diff.shape[1]))
print("The median, mean diff are {:.2f}, {:.2f}.".format(np.median(test_diff), np.mean(test_diff)))
snap = Image.fromarray(test_diff)
snap.show()

Got a difference of width, height as 300, 300.
The median, mean diff are 245.00, 166.01.


Apply thresholding to remove noise from motion and nuisance effects such as lighting change. In thresholding, we will set the value of a pixel in each of the RGB channels to 0 (min) or 255 (max) accordng to a binary threshold value. In OpenCV, this operation would be ```cv2.threshold(frame_delta, 50, 255, cv2.THRESH_BINARY)```. Use numpy operations here.

In [55]:
def threshold_difference(imdiff, threshold=50):
    return np.uint8(np.where(imdiff > threshold, 255, 0))

In [56]:
diff_clean = threshold_difference(test_diff, np.mean(test_diff))
print("Got a thresholded image of width, height as {}, {}.".format(diff_clean.shape[0], diff_clean.shape[1]))
diff_clean.dtype
snap = Image.fromarray(diff_clean)
snap.show()

Got a thresholded image of width, height as 300, 300.


Perform erosion and dilation operations to improve signal-to-noise. Apply the PIL methods ```minFilter()``` and ```MaxFilter()```for erosion and dilation respectively. Then, convert the image to black-and-white using the ```convert()``` method. All these methods are of the nature of image filters and operate on image as opposed to numpy array. Mutate accordingly. Get the result as a 2D numpy array.

In [57]:
def erode_dilate(snap, showme=False):
    snap_eroded = snap.filter(ImageFilter.MinFilter(7))
    if showme:
        snap_eroded.show()
        print("After erosion, got median, mean as {:.2f}, {:.2f}.".format(np.median(snap_eroded), np.mean(snap_eroded)))
    snap_dilated = snap_eroded.filter(ImageFilter.MaxFilter(3))
    if showme:
        snap_dilated.show()
        print("After dilation, got median, mean as {:.2f}, {:.2f}.".format(np.median(snap_dilated), np.mean(snap_dilated)))
    snap_bnw = snap_dilated.convert('1')
    if showme:
        snap_bnw.show()
        print("B&W image for motion detection has dimensions {}".format(bnw_array.shape))
    bnw_array = np.array(snap_bnw)
    return bnw_array

In [58]:
test_bnw = erode_dilate(snap)
test_bnw

array([[False, False, False, ...,  True,  True,  True],
       [False,  True, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       ...,
       [False, False, False, ...,  True, False,  True],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False,  True, False]])

Now that we havd 2D mask with connected regions, let us find the connected components, count them, then rank and sort them. This will give us an idea of whether there is motion in the image. We will use the algorithm in Aaron Becker's [instructional video](https://youtu.be/ticZclUYy88). 

This approach consists of two raster scans of the image. In the first scan, a foreground pixel is assigned a proposed membership label. As some of these labels may refer to the same connected component due to the nature of rastering, synonyms are noted and a second pass is made to update and finalze labels. Further details are explained in code.

In [59]:
test_pass = np.zeros(test_bnw.shape, dtype = np.int32)  # Mask 
count_foreground = 0
first_pass_counter = 0
synonyms = {}

for idx,x in np.ndenumerate(test_bnw):
    aboveme = 0     # B&W background
    leftofme = 0    # B&W background
    A = 0           # Label none
    B = 0           # Label none
    if x: # Not background
        """
        Is there a pixel above or to the left that is not background?
        Check and if found, obtain the numeric labels 
        marking connected components in 1st pass.
        """
        count_foreground += 1
        if (idx[0] > 0): # Yes, above
            aboveme = test_bnw[idx[0]-1, idx[1]]
            A = test_pass[idx[0]-1, idx[1]] # Get label
        if (idx[1] > 0): # Yes, on left
            leftofme = test_bnw[idx[0], idx[1]-1]
            B = test_pass[idx[0], idx[1]-1] # Get label
        """
        If both left and above have foreground,
        stick the lesser number as the label on our pixel.
        Note the conflict for 2nd pass correction.
        Note that if the lower value has already been marked
        for correction, follow the chain to the lowest value
        in the dictionary of synonymous labels.
        If only one of left or above have foreground,
        stick that label on our pixel.
        """
        if (A and B): # Contest           
            test_pass[idx] = min(A, B) # Resolve
            if (A != B): # Note for update in second pass
                if synonyms.get(min(A, B)):
                    synonyms[max(A, B)] = synonyms.get(min(A, B))
                else:
                    synonyms[max(A, B)] = min(A, B)
        elif A:
            test_pass[idx] = A
        elif B:
            test_pass[idx] = B        
        else:
            first_pass_counter += 1 # New label
            test_pass[idx] = first_pass_counter
            
print("Foreground has {} pixels and procesed {} in 1st pass.".format(sum(sum(test_bnw)), count_foreground))
print("First pass found {} candidates in first pass.".format(first_pass_counter))
Image.fromarray(test_pass).show()
print("Detected {} connected components.".format(len(set(synonyms.values()) )))

Foreground has 29067 pixels and procesed 29067 in 1st pass.
First pass found 3560 candidates in first pass.
Detected 96 connected components.


In [60]:
connected_components = {}
for idx,x in np.ndenumerate(test_pass):
    if x > 0: # Labeled
        label = synonyms.get(x, 0) 
        if label > 0: # Synonym found for label
            test_pass[idx] = label 
            connected_components[label] = connected_components.get(label, 0) + 1
        else:
            connected_components[x] = connected_components.get(x, 0) + 1
Image.fromarray(test_pass).show()

In [61]:
for label in sorted(connected_components, key=connected_components.get, reverse=True):
    print(label, connected_components[label])
# len(connected_components.keys())

17 16080
2381 1846
2792 1750
2422 1456
2565 948
3217 603
937 393
2222 351
1988 348
2362 320
3283 208
668 174
2058 115
1784 71
2586 69
1066 66
1033 63
3168 54
955 52
3115 49
750 45
3286 44
2394 39
6 27
3425 27
2981 26
2210 24
1854 23
3195 23
2298 19
2040 18
2363 17
9 16
2549 16
3463 16
2175 15
3277 15
150 13
2127 13
973 12
2292 12
1709 11
2356 11
2794 11
3025 11
2267 9
2988 9
3480 8
2370 7
895 6
1597 6
1920 6
2856 6
3302 6
66 5
1000 5
1546 5
1850 5
2004 5
2013 5
2018 5
2982 5
3059 5
3351 5
3384 5
47 4
328 4
929 4
1140 4
1188 4
1468 4
1543 4
2156 4
2471 4
2702 4
2798 4
2837 4
2997 4
3464 4
346 3
353 3
472 3
522 3
543 3
611 3
620 3
652 3
811 3
845 3
946 3
966 3
976 3
1044 3
1175 3
1274 3
1282 3
1322 3
1378 3
1428 3
1440 3
1476 3
1521 3
1595 3
1855 3
1950 3
1965 3
1997 3
2044 3
2121 3
2192 3
2260 3
2604 3
2648 3
2738 3
2803 3
2912 3
2915 3
2930 3
2944 3
2978 3
3041 3
3034 3
3163 3
3429 3
3438 3
3482 3
3491 3
8 2
11 2
13 2
15 2
24 2
30 2
49 2
111 2
134 2
260 2
270 2
283 2
314 2
320 2
347 2


Find connected regions.

## References:
1. A comprehensive DIY [guide](http://drsol.com/~deid/pi/camera/index.html) to Pi camera including many lesser-known techniques for image and video recording, processing and sharing.
2. A github [repo](https://gist.github.com/FutureSharks/ab4c22b719cdd894e3b7ffe1f5b8fd91) for pro motion detection with OpenCV.
3. A stackoverflow.com [post](https://stackoverflow.com/questions/31064974/whats-the-fastest-way-to-threshold-a-numpy-array) upon thresholding with operations upon numpy arrays.